In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Adrenocortical_Cancer/GSE19776'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Adrenocortical Carcinoma Gene Expression Profiling"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['Stage: NA', 'Stage: 2', 'Stage: 4', 'Stage: Recurrence', 'Stage: 3', 'Stage: 1', 'Stage: Unknown'], 1: ['tumor grade: NA', 'tumor grade: 3', 'tumor grade: 4', 'tumor grade: 2', 'tumor grade: 1', 'tumor grade: Unknown'], 2: ['functional: NA', 'functional: None', 'functional: Cushings', 'functional: Unknown', 'functional: aldosterone', 'functional: Virulizing', 'functional: Aldosterone', 'functional: Cortisol, aldosterone, testosterone'], 3: ['gender: Unknown', 'gender: M', 'gender: F', 'gender: NA'], 4: ['age in years: Unknown', 'age in years: 23.3', 'age in years: 56.5', 'age in years: 67.8', 'age in years: 72.1', 'age in years: 46.9', 'age in years: 26.7', 'age in years: 48.5', 'age in years: 36.9', 'age in years: 53.2', 'age i

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Sample characteristics provided as a dictionary (assumed from previous steps)
Sample_Characteristics_Dictionary = {
    0: ['tissue: adrenocortical carcinoma'],
    1: ['extent of disease: Localized', 'extent of disease: Metastatic', 'extent of disease: Regional', 'extent of disease: Unknown'],
    2: ['tumor grade: 4', 'tumor grade: 3', 'tumor grade: 2', 'tumor grade: 1', 'tumor grade: Unknown'],
    3: ['functional: None', 'functional: Cushings', 'functional: Unknown', 'functional: aldosterone'],
    4: ['gender: M', 'gender: F'],
    5: ['age: 67.8', 'age: 72.1', 'age: 26.7', 'age: 36.9', 'age: Unknown', 'age: 53.2', 'age: 37', 'age: 54.2', 'age: 67.3', 'age: 27.7', 'age: 58', 'age: 42', 'age: 46', 'age: 38'],
    6: ['survival in years: 1.6', 'survival in years: 0.4', 'survival in years: 10.4', 'survival in years: 8.1', 'survival in years: Unknown', 'survival in years: 7.4', 'survival in years: 6.8', 'survival in years: 0', 'survival in years: 5.3', 'survival in years: 11.9']
}

# Check if gene expression data is available
is_gene_available = True

# Determine the availability and keys for the required variables
trait_row = None
age_row = 5 if len({val.split(': ')[1] for val in Sample_Characteristics_Dictionary[5]}) > 1 else None
gender_row = 4 if len({val.split(': ')[1] for val in Sample_Characteristics_Dictionary[4]}) > 1 else None

# Define the conversion functions
def convert_trait(value):
    return None

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    gender_str = value.split(': ')[1]
    if gender_str == 'F':
        return 0
    elif gender_str == 'M':
        return 1
    else:
        return None

# Save cohort information
save_cohort_info('GSE19776', './preprocessed/Adrenocortical_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Adrenocortical_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Adrenocortical_Cancer/trait_data/GSE19776.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
